# BMIの欠損値を補充する

In [1]:
import pandas as pd 
import numpy as np
import matplotlib as plt
import japanize_matplotlib 
import os 
import openpyxl

In [2]:
cd "E:/"

E:\


In [3]:
path = "2021_12_16_100834_ExportFile/"
df = pd.read_csv(path + "a591889084094fb8a64fff2a0b11eb67__EDFU_肥満の状態.csv" , sep = ',' , encoding = "Shift_JIS")

# 除外症例の読み込み
exception = pd.read_excel("★除外症例（テスト症例）_匿名化IDのみ.xlsx")

In [4]:
df

,施設匿名化ID,作成日,エントリー回数,ObjectHash,データ種別,収縮期血圧（SBP）,拡張期血圧（DBP）,脈拍（PULSE）,実測体重（BWA）,申告体重（BWR）,身長（BH）,腹囲（WAIST）,実測BMI（BMIA）,申告BMI（BMIR）,CTによる内蔵脂肪型肥満の確認（VOB）,内臓脂肪面積（VFA）,登録施設ID,登録ユーザID
0,KOB0000005,2019/02/22,1,45de99b2-31fd-4f9a-8618-845a2f9de36c,FB,NaN,NaN,NaN,64.0,NaN,149.9,NaN,28.48,NaN,NaN,NaN,JOKOB,JOKOBAdmin
1,KOB0000005,2019/06/24,2,45de99b2-31fd-4f9a-8618-845a2f9de36c,FB,NaN,NaN,NaN,NaN,NaN,149.9,NaN,NaN,NaN,20.0,NaN,JOKOB,JOKOBAdmin
2,KOB0000005,2019/06/25,3,45de99b2-31fd-4f9a-8618-845a2f9de36c,FB,NaN,NaN,NaN,NaN,NaN,149.9,NaN,NaN,NaN,20.0,NaN,JOKOB,JOKOBAdmin
3,KOB0000005,2019/09/10,4,45de99b2-31fd-4f9a-8618-845a2f9de36c,FB,NaN,NaN,NaN,NaN,NaN,149.9,NaN,NaN,NaN,20.0,NaN,JOKOB,JOKOBAdmin
4,KOB0000005,2019/11/12,5,45de99b2-31fd-4f9a-8618-845a2f9de36c,FB,NaN,NaN,NaN,NaN,NaN,149.9,NaN,NaN,NaN,20.0,NaN,JOKOB,JOKOBAdmin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11361,KOB0000971,2021/11/01,2,a991d2d3-02c4-4c44-bfb4-5249551faa91,FB,NaN,NaN,NaN,NaN,NaN,163.0,109.0,NaN,NaN,20.0,NaN,JOKOB,JOKOBAdmin
11362,KOB0000972,2021/10/07,1,1779fb70-1847-43ce-8dd0-d3da8cec6bec,FB,117.0,78.0,NaN,NaN,NaN,168.0,91.0,NaN,NaN,20.0,NaN,JOKOB,JOKOBAdmin
11363,KOB0000972,2021/11/04,2,1779fb70-1847-43ce-8dd0-d3da8cec6bec,FB,NaN,NaN,NaN,NaN,NaN,168.0,NaN,NaN,NaN,20.0,NaN,JOKOB,JOKOBAdmin
11364,KOB0000974,2021/10/28,1,0740c121-755a-42d3-8cbc-2eb236b67106,FB,NaN,NaN,NaN,NaN,88.0,158.0,NaN,NaN,35.25,NaN,NaN,JOKOB,JOKOBAdmin


In [5]:
df.columns

Index(['施設匿名化ID', '作成日', 'エントリー回数', 'ObjectHash', 'データ種別', '収縮期血圧（SBP）',
       '拡張期血圧（DBP）', '脈拍（PULSE）', '実測体重（BWA）', '申告体重（BWR）', '身長（BH）',
       '腹囲（WAIST）', '実測BMI（BMIA）', '申告BMI（BMIR）', 'CTによる内蔵脂肪型肥満の確認（VOB）',
       '内臓脂肪面積（VFA）', '登録施設ID', '登録ユーザID'],
      dtype='object')

In [6]:
df = df[['施設匿名化ID', '作成日',  '収縮期血圧（SBP）',
       '拡張期血圧（DBP）', '脈拍（PULSE）', '実測体重（BWA）', '申告体重（BWR）', '身長（BH）',
       '腹囲（WAIST）', '実測BMI（BMIA）', '申告BMI（BMIR）', 'CTによる内蔵脂肪型肥満の確認（VOB）',
       '内臓脂肪面積（VFA）']]

In [7]:
df.columns = ['ID', '作成日', 'systole', 'diastole', 'HR', 'weight_real',
       'weight_say', 'height', 'waist', 'BMI_real', 'BMI_say',
       'CTによる内臓脂肪型肥満確認', 'VFA']

In [8]:
df.head()

,ID,作成日,systole,diastole,HR,weight_real,weight_say,height,waist,BMI_real,BMI_say,CTによる内臓脂肪型肥満確認,VFA
0,KOB0000005,2019/02/22,NaN,NaN,NaN,64.0,NaN,149.9,NaN,28.48,NaN,NaN,NaN
1,KOB0000005,2019/06/24,NaN,NaN,NaN,NaN,NaN,149.9,NaN,NaN,NaN,20.0,NaN
2,KOB0000005,2019/06/25,NaN,NaN,NaN,NaN,NaN,149.9,NaN,NaN,NaN,20.0,NaN
3,KOB0000005,2019/09/10,NaN,NaN,NaN,NaN,NaN,149.9,NaN,NaN,NaN,20.0,NaN
4,KOB0000005,2019/11/12,NaN,NaN,NaN,NaN,NaN,149.9,NaN,NaN,NaN,20.0,NaN


In [9]:
exception["匿名ID"].unique()

array(['KOB0000004', 'KOB0000719', 'KOB0000720', 'KOB0000750',
       'KOB0000751'], dtype=object)

In [10]:
# 除外症例を除く
df = df.set_index("ID")
# 欠損した番号を振り直すために2回インデックスをリセットする。
df = df.drop(index = ['KOB0000004' ,'KOB0000719','KOB0000720', 'KOB0000750'] ).reset_index().reset_index(drop = True)

In [394]:
df.head()

,ID,作成日,systole,diastole,HR,weight_real,weight_say,height,waist,BMI_real,BMI_say,CTによる内臓脂肪型肥満確認,VFA
0,KOB0000005,2019/02/22,NaN,NaN,NaN,64.0,NaN,149.9,NaN,28.48,NaN,NaN,NaN
1,KOB0000005,2019/06/24,NaN,NaN,NaN,NaN,NaN,149.9,NaN,NaN,NaN,20.0,NaN
2,KOB0000005,2019/06/25,NaN,NaN,NaN,NaN,NaN,149.9,NaN,NaN,NaN,20.0,NaN
3,KOB0000005,2019/09/10,NaN,NaN,NaN,NaN,NaN,149.9,NaN,NaN,NaN,20.0,NaN
4,KOB0000005,2019/11/12,NaN,NaN,NaN,NaN,NaN,149.9,NaN,NaN,NaN,20.0,NaN


In [395]:
# 体重からBMI計算
df["BMI_real_cal"] = df.weight_real / (df.height  / 100)**2
df["BMI_say_cal"] = df.weight_say   / (df.height / 100)**2

In [396]:
#有効数字に丸める
df["BMI_real_cal"] = df["BMI_real_cal"].map(lambda x : round(x , 2))
df["BMI_say_cal"] = df["BMI_say_cal"].map(lambda x : round(x , 2))

In [397]:
df.head()

,ID,作成日,systole,diastole,HR,weight_real,weight_say,height,waist,BMI_real,BMI_say,CTによる内臓脂肪型肥満確認,VFA,BMI_real_cal,BMI_say_cal
0,KOB0000005,2019/02/22,NaN,NaN,NaN,64.0,NaN,149.9,NaN,28.48,NaN,NaN,NaN,28.48,NaN
1,KOB0000005,2019/06/24,NaN,NaN,NaN,NaN,NaN,149.9,NaN,NaN,NaN,20.0,NaN,NaN,NaN
2,KOB0000005,2019/06/25,NaN,NaN,NaN,NaN,NaN,149.9,NaN,NaN,NaN,20.0,NaN,NaN,NaN
3,KOB0000005,2019/09/10,NaN,NaN,NaN,NaN,NaN,149.9,NaN,NaN,NaN,20.0,NaN,NaN,NaN
4,KOB0000005,2019/11/12,NaN,NaN,NaN,NaN,NaN,149.9,NaN,NaN,NaN,20.0,NaN,NaN,NaN


In [398]:
# cal と既存のBMIのうち最大値を選択する
real = df[["BMI_real" , "BMI_real_cal"]].max(axis = 1 )
say = df[["BMI_say" , "BMI_say_cal"]].max(axis = 1 )

In [399]:
# BMIの欠損値を補充する
df["BMI_real"] = real
df["BMI_say"] = say

In [400]:
# weigh_real優先でweigh_sayも組み込んだ列weight_eitherを作成する。
# realが0より大きいならrealを、そうでないならsayを入れる。
df["weight_either"] = 0
for i in np.arange(len(df)):
    if df.weight_real[i] >0:
        df["weight_either"][i] = df["weight_real"][i]
    else:
        df["weight_either"][i] = df["weight_say"][i]

C:\Users\Yoshi\AppData\Local\Temp\ipykernel_1644\1982366703.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["weight_either"][i] = df["weight_real"][i]
C:\Users\Yoshi\AppData\Local\Temp\ipykernel_1644\1982366703.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["weight_either"][i] = df["weight_say"][i]


In [401]:
# weigh_real優先でweigh_sayも組み込んだ列weight_eitherを作成する。
# realが0より大きいならrealを、そうでないならsayを入れる。
df["BMI_either"] = 0
for i in np.arange(len(df)):
    if df.weight_real[i] >0:
        df["BMI_either"][i] = df["BMI_real"][i]
    else:
        df["BMI_either"][i] = df["BMI_say"][i]

C:\Users\Yoshi\AppData\Local\Temp\ipykernel_1644\2017303668.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["BMI_either"][i] = df["BMI_real"][i]


In [402]:
# 順番を並べ替える
df = df[['ID', '作成日', 'weight_real', 'weight_say', 'weight_either' , 
       'BMI_real', 'BMI_say','BMI_either' ,'systole', 'diastole', 'HR', 
       'CTによる内臓脂肪型肥満確認', 'VFA', 'height', 'waist']]
df.head()

# とりあえずここまででBMIのデータ補充は完了

,ID,作成日,weight_real,weight_say,weight_either,BMI_real,BMI_say,BMI_either,systole,diastole,HR,CTによる内臓脂肪型肥満確認,VFA,height,waist
0,KOB0000005,2019/02/22,64.0,NaN,64.0,28.48,NaN,28.48,NaN,NaN,NaN,NaN,NaN,149.9,NaN
1,KOB0000005,2019/06/24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,149.9,NaN
2,KOB0000005,2019/06/25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,149.9,NaN
3,KOB0000005,2019/09/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,149.9,NaN
4,KOB0000005,2019/11/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,149.9,NaN


In [403]:
# 一度savefolderに保存する
df.to_csv("E:/savefolder/肥満の状態_欠損値補充.csv")

# 異常値の除去

In [404]:
# BMIの降順でおかしいのを消す
df.sort_values(by = "BMI_either" , ascending= False).head(8)




,ID,作成日,weight_real,weight_say,weight_either,BMI_real,BMI_say,BMI_either,systole,diastole,HR,CTによる内臓脂肪型肥満確認,VFA,height,waist
8284,KOB0000572,2020/01/31,NaN,702.0,702.0,NaN,296.00,296.00,118.0,60.0,NaN,20.0,NaN,154.0,97.0
2268,KOB0000159,2019/12/09,473.0,NaN,473.0,173.95,NaN,173.95,120.0,70.0,80.0,20.0,NaN,164.9,NaN
10494,KOB0000789,2021/01/14,NaN,70.0,70.0,NaN,155.94,155.94,NaN,NaN,NaN,20.0,NaN,67.0,NaN
10490,KOB0000789,2020/12/17,NaN,70.0,70.0,NaN,155.94,155.94,NaN,NaN,NaN,20.0,NaN,67.0,NaN
8904,KOB0000623,2019/06/25,NaN,144.0,144.0,NaN,60.17,60.17,NaN,NaN,NaN,NaN,NaN,154.7,NaN
8907,KOB0000623,2019/06/25,NaN,144.0,144.0,NaN,60.17,60.17,NaN,NaN,NaN,NaN,NaN,154.7,NaN
1018,KOB0000076,2019/02/25,153.7,NaN,153.7,60.04,NaN,60.04,NaN,NaN,NaN,NaN,NaN,160.0,NaN
1019,KOB0000076,2019/02/25,153.7,NaN,153.7,60.04,NaN,60.04,NaN,NaN,NaN,NaN,NaN,160.0,NaN


In [405]:
# 2268 , 8284 ,10490 ,10494　を消す
df.drop(index = [2268 , 8284 ,10490 ,10494] , inplace= True)

#####################　BMIの昇順に関しては解釈難しそうなので後で一つずつ確認して処理する ######################################

C:\Users\Yoshi\AppData\Local\Temp\ipykernel_1644\3281265301.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(index = [2268 , 8284 ,10490 ,10494] , inplace= True)


In [406]:
# 収縮期血圧降順でおかしいやつ消す。
df.sort_values(by = "systole" , ascending= False).head(8)


,ID,作成日,weight_real,weight_say,weight_either,BMI_real,BMI_say,BMI_either,systole,diastole,HR,CTによる内臓脂肪型肥満確認,VFA,height,waist
910,KOB0000066,2020/05/25,81.4,NaN,81.4,28.17,NaN,28.17,986.0,NaN,NaN,20.0,NaN,170.0,NaN
2672,KOB0000187,2020/04/13,NaN,67.0,67.0,NaN,26.17,26.17,177.0,84.0,NaN,20.0,NaN,160.0,NaN
5535,KOB0000365,2020/02/18,NaN,NaN,NaN,NaN,NaN,NaN,175.0,89.0,100.0,20.0,NaN,180.0,NaN
8300,KOB0000573,2020/02/05,NaN,53.6,53.6,NaN,23.82,23.82,171.0,69.0,NaN,NaN,NaN,150.0,84.5
5693,KOB0000377,2019/02/13,NaN,65.5,65.5,NaN,27.98,27.98,170.0,90.0,NaN,NaN,NaN,153.0,105.0
2808,KOB0000196,2019/10/28,67.9,NaN,67.9,26.86,NaN,26.86,170.0,90.0,NaN,20.0,NaN,159.0,NaN
2017,KOB0000141,2021/06/10,75.5,NaN,75.5,32.85,NaN,32.85,169.0,85.0,NaN,20.0,NaN,151.6,NaN
9522,KOB0000675,2019/08/07,NaN,55.0,55.0,NaN,24.44,24.44,169.0,90.0,NaN,NaN,NaN,150.0,NaN


In [407]:
# 910を消す
df.drop(index = 910 , inplace= True)

C:\Users\Yoshi\AppData\Local\Temp\ipykernel_1644\2895199075.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(index = 910 , inplace= True)


In [408]:
# 拡張期血圧降順でおかしいやつ消す。
df.sort_values(by = "diastole" , ascending= True).head(10)

,ID,作成日,weight_real,weight_say,weight_either,BMI_real,BMI_say,BMI_either,systole,diastole,HR,CTによる内臓脂肪型肥満確認,VFA,height,waist
7468,KOB0000493,2019/07/08,144.8,NaN,144.8,44.00,NaN,44.00,142.0,8.0,87.0,20.0,NaN,181.4,NaN
7461,KOB0000493,2019/07/08,144.8,NaN,144.8,44.00,NaN,44.00,142.0,8.0,87.0,20.0,NaN,181.4,NaN
1486,KOB0000108,2021/09/08,NaN,NaN,NaN,NaN,NaN,NaN,95.0,25.0,73.0,20.0,NaN,158.1,NaN
7258,KOB0000478,2021/02/09,NaN,73.0,73.0,NaN,32.44,32.44,120.0,40.0,NaN,20.0,NaN,150.0,NaN
8327,KOB0000576,2019/06/24,63.9,NaN,63.9,25.60,NaN,25.60,90.0,40.0,NaN,NaN,NaN,158.0,158.0
7257,KOB0000478,2020/10/01,NaN,70.0,70.0,NaN,31.11,31.11,120.0,40.0,NaN,20.0,NaN,150.0,NaN
8328,KOB0000576,2019/08/26,63.2,NaN,63.2,25.32,NaN,25.32,110.0,40.0,98.0,NaN,NaN,158.0,NaN
6230,KOB0000411,2019/02/14,NaN,68.2,68.2,NaN,25.67,25.67,132.0,40.0,NaN,NaN,NaN,163.0,NaN
8332,KOB0000576,2019/06/24,63.9,NaN,63.9,25.60,NaN,25.60,90.0,40.0,NaN,NaN,NaN,158.0,158.0
8333,KOB0000576,2019/08/26,63.2,NaN,63.2,25.32,NaN,25.32,110.0,40.0,98.0,NaN,NaN,158.0,NaN


In [409]:
# 7468 , 7461 , 1486を消す
df.drop(index = [7468 , 7461 , 1486] , inplace= True)
# 明らかな異常値のみ消去し、その他のデータは削除しなかった。

In [410]:
# HRの降順でソート
df.sort_values(by = "HR" , ascending= False).head(8)

,ID,作成日,weight_real,weight_say,weight_either,BMI_real,BMI_say,BMI_either,systole,diastole,HR,CTによる内臓脂肪型肥満確認,VFA,height,waist
5003,KOB0000328,2021/08/11,NaN,NaN,NaN,NaN,NaN,NaN,117.0,62.0,787.0,10.0,79.4,170.0,NaN
887,KOB0000065,2020/04/07,97.6,NaN,97.6,31.87,NaN,31.87,98.0,48.0,128.0,20.0,NaN,175.0,NaN
5265,KOB0000348,2021/03/24,77.5,NaN,77.5,26.88,NaN,26.88,100.0,60.0,124.0,20.0,NaN,169.8,NaN
7881,KOB0000532,2019/07/22,75.7,NaN,75.7,29.24,NaN,29.24,134.0,80.0,120.0,NaN,NaN,160.9,NaN
7876,KOB0000532,2019/07/22,75.7,NaN,75.7,29.24,NaN,29.24,134.0,80.0,120.0,NaN,NaN,160.9,NaN
4415,KOB0000292,2021/07/21,NaN,NaN,NaN,NaN,NaN,NaN,134.0,79.0,119.0,20.0,NaN,154.0,NaN
4404,KOB0000292,2019/12/25,NaN,57.0,57.0,NaN,24.03,24.03,144.0,75.0,117.0,20.0,NaN,154.0,NaN
4413,KOB0000292,2021/05/26,NaN,NaN,NaN,NaN,NaN,NaN,148.0,81.0,114.0,20.0,NaN,154.0,NaN


In [411]:
# 5003を消す
df.drop(index = 5003 , inplace= True)

In [412]:
# heigh昇順でソート
df.sort_values(by = "height" , ascending= True).head(8)

,ID,作成日,weight_real,weight_say,weight_either,BMI_real,BMI_say,BMI_either,systole,diastole,HR,CTによる内臓脂肪型肥満確認,VFA,height,waist
10488,KOB0000789,2020/10/01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,67.0,NaN
10486,KOB0000789,2020/07/01,NaN,NaN,NaN,NaN,NaN,NaN,130.0,90.0,NaN,20.0,NaN,67.0,NaN
10487,KOB0000789,2020/09/03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,67.0,89.0
10496,KOB0000789,2021/03/25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,67.0,NaN
10495,KOB0000789,2021/02/25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,67.0,NaN
10489,KOB0000789,2020/11/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,67.0,NaN
10631,KOB0000804,2021/05/06,NaN,56.0,56.0,NaN,30.28,30.28,126.0,84.0,84.0,20.0,NaN,136.0,NaN
10633,KOB0000804,2021/07/15,NaN,NaN,NaN,NaN,NaN,NaN,110.0,66.0,80.0,20.0,NaN,136.0,NaN


In [413]:
# 10488 , 10486 , 10487 , 10496 , 10495 , 10489 を消す
df.drop(index = [10488 , 10486 , 10487 , 10496 , 10495 , 10489] , inplace= True)

In [414]:
df.sort_values(by = "BMI_either" , ascending= True).head(5)

,ID,作成日,weight_real,weight_say,weight_either,BMI_real,BMI_say,BMI_either,systole,diastole,HR,CTによる内臓脂肪型肥満確認,VFA,height,waist
1581,KOB0000113,2021/10/01,NaN,0.87,0.87,NaN,0.37,0.37,118.0,68.0,NaN,20.0,NaN,154.0,NaN
9507,KOB0000674,2019/12/04,NaN,37.00,37.00,NaN,15.40,15.40,119.0,76.0,67.0,NaN,NaN,155.0,64.0
1550,KOB0000112,2020/07/13,NaN,36.00,36.00,NaN,15.56,15.56,NaN,NaN,NaN,20.0,NaN,152.1,NaN
8603,KOB0000598,2019/08/07,NaN,42.00,42.00,NaN,15.81,15.81,110.0,60.0,NaN,NaN,NaN,163.0,55.0
8605,KOB0000598,2020/02/05,NaN,42.00,42.00,NaN,15.81,15.81,110.0,60.0,NaN,NaN,NaN,163.0,55.0


In [415]:
# 1581はおかしいので消す BMI 0.37 はさすがに...
df.drop(index = 1581 , inplace= True)

## BMIが一度も25を超えていない症例のIDをメモする

In [416]:
# 一つずつ処理していくのはキリがないため一度もBMIが25以上になったことがない人を除外する
# BMIで昇順に直したデータを作る
BMI_acc_max_groupby_ID = pd.DataFrame( df.groupby(by = "ID")["BMI_either"].max().sort_values(ascending= True) )

In [417]:
# BMI_eitherが欠損値しか存在しないIDを保存する
idx = BMI_acc_max_groupby_ID["BMI_either"] > 0 
idx = idx == False
defect = BMI_acc_max_groupby_ID[idx]

In [418]:
defect.head()

,BMI_either
ID,
KOB0000562,NaN
KOB0000570,NaN
KOB0000588,NaN
KOB0000592,NaN
KOB0000603,NaN


In [419]:
# CSVとして保存
defect.to_csv("E:/savefolder/BMI_一度も入力されてないID.csv")


In [420]:
# BMI_eitherが最大でも25以下しかないIDを保存する
idx = BMI_acc_max_groupby_ID["BMI_either"] < 25
BMI_lower_25 = BMI_acc_max_groupby_ID[idx]
BMI_lower_25.head()

,BMI_either
ID,
KOB0000721,16.02
KOB0000598,16.37
KOB0000674,16.65
KOB0000683,17.31
KOB0000602,18.00


In [421]:
print(len(defect))
print(len(BMI_lower_25))

56
56


In [422]:
#  CSVとして保存
BMI_lower_25.to_csv("E:/savefolder/BMI_25以下しかないID.csv")

## BMI最大値が25以下の症例と一度も入っていない症例のIDを取得して、dropメゾットでdfから落としていく

In [423]:
BMI_lower_25.index

Index(['KOB0000721', 'KOB0000598', 'KOB0000674', 'KOB0000683', 'KOB0000602',
       'KOB0000738', 'KOB0000630', 'KOB0000612', 'KOB0000616', 'KOB0000574',
       'KOB0000577', 'KOB0000621', 'KOB0000686', 'KOB0000600', 'KOB0000599',
       'KOB0000737', 'KOB0000637', 'KOB0000679', 'KOB0000589', 'KOB0000595',
       'KOB0000660', 'KOB0000727', 'KOB0000646', 'KOB0000608', 'KOB0000614',
       'KOB0000688', 'KOB0000729', 'KOB0000645', 'KOB0000802', 'KOB0000593',
       'KOB0000644', 'KOB0000740', 'KOB0000656', 'KOB0000618', 'KOB0000605',
       'KOB0000725', 'KOB0000739', 'KOB0000596', 'KOB0000657', 'KOB0000906',
       'KOB0000597', 'KOB0000573', 'KOB0000954', 'KOB0000950', 'KOB0000675',
       'KOB0000941', 'KOB0000569', 'KOB0000723', 'KOB0000578', 'KOB0000131',
       'KOB0000581', 'KOB0000568', 'KOB0000633', 'KOB0000960', 'KOB0000778',
       'KOB0000707'],
      dtype='object', name='ID')

In [424]:
defect.index

Index(['KOB0000562', 'KOB0000570', 'KOB0000588', 'KOB0000592', 'KOB0000603',
       'KOB0000604', 'KOB0000610', 'KOB0000634', 'KOB0000638', 'KOB0000648',
       'KOB0000654', 'KOB0000655', 'KOB0000658', 'KOB0000677', 'KOB0000685',
       'KOB0000690', 'KOB0000696', 'KOB0000708', 'KOB0000711', 'KOB0000713',
       'KOB0000715', 'KOB0000718', 'KOB0000734', 'KOB0000744', 'KOB0000747',
       'KOB0000760', 'KOB0000762', 'KOB0000765', 'KOB0000771', 'KOB0000803',
       'KOB0000815', 'KOB0000819', 'KOB0000821', 'KOB0000824', 'KOB0000828',
       'KOB0000830', 'KOB0000831', 'KOB0000835', 'KOB0000838', 'KOB0000840',
       'KOB0000846', 'KOB0000876', 'KOB0000878', 'KOB0000880', 'KOB0000898',
       'KOB0000907', 'KOB0000910', 'KOB0000913', 'KOB0000929', 'KOB0000934',
       'KOB0000957', 'KOB0000958', 'KOB0000962', 'KOB0000966', 'KOB0000970',
       'KOB0000972'],
      dtype='object', name='ID')

In [425]:
idx_low = ['KOB0000721', 'KOB0000598', 'KOB0000674', 'KOB0000683', 'KOB0000602',
       'KOB0000738', 'KOB0000630', 'KOB0000612', 'KOB0000616', 'KOB0000574',
       'KOB0000577', 'KOB0000621', 'KOB0000686', 'KOB0000600', 'KOB0000599',
       'KOB0000737', 'KOB0000637', 'KOB0000679', 'KOB0000589', 'KOB0000595',
       'KOB0000660', 'KOB0000727', 'KOB0000646', 'KOB0000608', 'KOB0000614',
       'KOB0000688', 'KOB0000729', 'KOB0000645', 'KOB0000802', 'KOB0000593',
       'KOB0000644', 'KOB0000740', 'KOB0000656', 'KOB0000618', 'KOB0000605',
       'KOB0000725', 'KOB0000739', 'KOB0000596', 'KOB0000657', 'KOB0000906',
       'KOB0000597', 'KOB0000573', 'KOB0000954', 'KOB0000950', 'KOB0000675',
       'KOB0000941', 'KOB0000569', 'KOB0000723', 'KOB0000578', 'KOB0000131',
       'KOB0000581', 'KOB0000568', 'KOB0000633', 'KOB0000960', 'KOB0000778',
       'KOB0000707']

idx_defect = ['KOB0000562', 'KOB0000570', 'KOB0000588', 'KOB0000592', 'KOB0000603',
       'KOB0000604', 'KOB0000610', 'KOB0000634', 'KOB0000638', 'KOB0000648',
       'KOB0000654', 'KOB0000655', 'KOB0000658', 'KOB0000677', 'KOB0000685',
       'KOB0000690', 'KOB0000696', 'KOB0000708', 'KOB0000711', 'KOB0000713',
       'KOB0000715', 'KOB0000718', 'KOB0000734', 'KOB0000744', 'KOB0000747',
       'KOB0000760', 'KOB0000762', 'KOB0000765', 'KOB0000771', 'KOB0000803',
       'KOB0000815', 'KOB0000819', 'KOB0000821', 'KOB0000824', 'KOB0000828',
       'KOB0000830', 'KOB0000831', 'KOB0000835', 'KOB0000838', 'KOB0000840',
       'KOB0000846', 'KOB0000876', 'KOB0000878', 'KOB0000880', 'KOB0000898',
       'KOB0000907', 'KOB0000910', 'KOB0000913', 'KOB0000929', 'KOB0000934',
       'KOB0000957', 'KOB0000958', 'KOB0000962', 'KOB0000966', 'KOB0000970',
       'KOB0000972']

ここから更に、dfからこの2つに該当するデータを除く  
str.contain　を使用するために一度、インデックスを文字列に直した後に|の演算子を使用する

In [426]:
# 最初に文字列の長さを調べる
len(df)

11322

In [427]:
str(idx_low)

"['KOB0000721', 'KOB0000598', 'KOB0000674', 'KOB0000683', 'KOB0000602', 'KOB0000738', 'KOB0000630', 'KOB0000612', 'KOB0000616', 'KOB0000574', 'KOB0000577', 'KOB0000621', 'KOB0000686', 'KOB0000600', 'KOB0000599', 'KOB0000737', 'KOB0000637', 'KOB0000679', 'KOB0000589', 'KOB0000595', 'KOB0000660', 'KOB0000727', 'KOB0000646', 'KOB0000608', 'KOB0000614', 'KOB0000688', 'KOB0000729', 'KOB0000645', 'KOB0000802', 'KOB0000593', 'KOB0000644', 'KOB0000740', 'KOB0000656', 'KOB0000618', 'KOB0000605', 'KOB0000725', 'KOB0000739', 'KOB0000596', 'KOB0000657', 'KOB0000906', 'KOB0000597', 'KOB0000573', 'KOB0000954', 'KOB0000950', 'KOB0000675', 'KOB0000941', 'KOB0000569', 'KOB0000723', 'KOB0000578', 'KOB0000131', 'KOB0000581', 'KOB0000568', 'KOB0000633', 'KOB0000960', 'KOB0000778', 'KOB0000707']"

In [428]:
# 一度文字列化
str(idx_low).replace("," , "|").replace(" " , "").replace("'" , "")

'[KOB0000721|KOB0000598|KOB0000674|KOB0000683|KOB0000602|KOB0000738|KOB0000630|KOB0000612|KOB0000616|KOB0000574|KOB0000577|KOB0000621|KOB0000686|KOB0000600|KOB0000599|KOB0000737|KOB0000637|KOB0000679|KOB0000589|KOB0000595|KOB0000660|KOB0000727|KOB0000646|KOB0000608|KOB0000614|KOB0000688|KOB0000729|KOB0000645|KOB0000802|KOB0000593|KOB0000644|KOB0000740|KOB0000656|KOB0000618|KOB0000605|KOB0000725|KOB0000739|KOB0000596|KOB0000657|KOB0000906|KOB0000597|KOB0000573|KOB0000954|KOB0000950|KOB0000675|KOB0000941|KOB0000569|KOB0000723|KOB0000578|KOB0000131|KOB0000581|KOB0000568|KOB0000633|KOB0000960|KOB0000778|KOB0000707]'

In [429]:
# 一度文字列化
low = "KOB0000721|KOB0000598|KOB0000674|KOB0000683|KOB0000602|KOB0000738|KOB0000630|KOB0000612|KOB0000616|KOB0000574|KOB0000577|KOB0000621|KOB0000686|KOB0000600|KOB0000599|KOB0000737|KOB0000637|KOB0000679|KOB0000589|KOB0000595|KOB0000660|KOB0000727|KOB0000646|KOB0000608|KOB0000614|KOB0000688|KOB0000729|KOB0000645|KOB0000802|KOB0000593|KOB0000644|KOB0000740|KOB0000656|KOB0000618|KOB0000605|KOB0000725|KOB0000739|KOB0000596|KOB0000657|KOB0000906|KOB0000597|KOB0000573|KOB0000954|KOB0000950|KOB0000675|KOB0000941|KOB0000569|KOB0000723|KOB0000578|KOB0000131|KOB0000581|KOB0000568|KOB0000633|KOB0000960|KOB0000778|KOB0000707"

In [430]:
# 25以下しか入力されていないもののbool値
# それをさらに反転させることで得たいインデックスを得る
idx_1 = df["ID"].str.contains(low, na = True) == False
idx_1

#　idx_1を満たすdfのみを抽出
df = df[idx_1]

In [431]:
len(df)

10757

In [432]:
str(idx_defect).replace("," , "|").replace(" " , "").replace("'" , "")

'[KOB0000562|KOB0000570|KOB0000588|KOB0000592|KOB0000603|KOB0000604|KOB0000610|KOB0000634|KOB0000638|KOB0000648|KOB0000654|KOB0000655|KOB0000658|KOB0000677|KOB0000685|KOB0000690|KOB0000696|KOB0000708|KOB0000711|KOB0000713|KOB0000715|KOB0000718|KOB0000734|KOB0000744|KOB0000747|KOB0000760|KOB0000762|KOB0000765|KOB0000771|KOB0000803|KOB0000815|KOB0000819|KOB0000821|KOB0000824|KOB0000828|KOB0000830|KOB0000831|KOB0000835|KOB0000838|KOB0000840|KOB0000846|KOB0000876|KOB0000878|KOB0000880|KOB0000898|KOB0000907|KOB0000910|KOB0000913|KOB0000929|KOB0000934|KOB0000957|KOB0000958|KOB0000962|KOB0000966|KOB0000970|KOB0000972]'

In [433]:
# 一度文字列化
defect = "KOB0000562|KOB0000570|KOB0000588|KOB0000592|KOB0000603|KOB0000604|KOB0000610|KOB0000634|KOB0000638|KOB0000648|KOB0000654|KOB0000655|KOB0000658|KOB0000677|KOB0000685|KOB0000690|KOB0000696|KOB0000708|KOB0000711|KOB0000713|KOB0000715|KOB0000718|KOB0000734|KOB0000744|KOB0000747|KOB0000760|KOB0000762|KOB0000765|KOB0000771|KOB0000803|KOB0000815|KOB0000819|KOB0000821|KOB0000824|KOB0000828|KOB0000830|KOB0000831|KOB0000835|KOB0000838|KOB0000840|KOB0000846|KOB0000876|KOB0000878|KOB0000880|KOB0000898|KOB0000907|KOB0000910|KOB0000913|KOB0000929|KOB0000934|KOB0000957|KOB0000958|KOB0000962|KOB0000966|KOB0000970|KOB0000972"

In [434]:
# BMIが全く入力されていないもののbool値
# それをさらに反転させることで得たいインデックスを得る
idx_2 = df["ID"].str.contains(defect, na = True) == False
idx_2

#　idx_1を満たすdfのみを抽出
df = df[idx_2]

In [435]:
df

,ID,作成日,weight_real,weight_say,weight_either,BMI_real,BMI_say,BMI_either,systole,diastole,HR,CTによる内臓脂肪型肥満確認,VFA,height,waist
0,KOB0000005,2019/02/22,64.0,NaN,64.0,28.48,NaN,28.48,NaN,NaN,NaN,NaN,NaN,149.9,NaN
1,KOB0000005,2019/06/24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,149.9,NaN
2,KOB0000005,2019/06/25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,149.9,NaN
3,KOB0000005,2019/09/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,149.9,NaN
4,KOB0000005,2019/11/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,149.9,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11329,KOB0000969,2021/11/08,NaN,76.8,76.8,NaN,29.96,29.96,NaN,NaN,NaN,10.0,32.8,160.1,NaN
11332,KOB0000971,2021/10/25,NaN,85.0,85.0,NaN,31.99,31.99,NaN,NaN,NaN,20.0,NaN,163.0,NaN
11333,KOB0000971,2021/11/01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,163.0,109.0
11336,KOB0000974,2021/10/28,NaN,88.0,88.0,NaN,35.25,35.25,NaN,NaN,NaN,NaN,NaN,158.0,NaN


In [436]:
len(df)
# 800レコードくらい減っちゃった。

10516

In [437]:
# CSVとして保存
df.to_csv("E:/savefolder/BMI_異常値など排除.csv")